<a href="https://colab.research.google.com/github/2303a51la3/Sathvika_A3_NLPLAB-B_30-/blob/main/lab9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import to_categorical

# Parameters
batch_size = 64
epochs = 100
latent_dim = 256
input_vocab_size = 100  # adjust based on actual vocab size
target_vocab_size = 100  # adjust based on actual vocab size

# Sample data (use your own dataset)
data = [
    ("hello", "bonjour"),
    ("how are you", "comment ça va"),
    ("I am fine", "je vais bien"),
    ("what is your name", "comment tu t'appelles"),
    ("my name is", "je m'appelle"),
    ("thank you", "merci"),
    ("goodbye", "au revoir")
]

# Vocabulary creation
input_texts = [pair[0] for pair in data]
target_texts = [pair[1] for pair in data]

# Create a mapping of unique tokens for both input and target sequences
input_tokens = sorted(set(" ".join(input_texts).split()))
target_tokens = sorted(set(" ".join(target_texts).split()))

input_token_index = dict([(char, i + 1) for i, char in enumerate(input_tokens)])
target_token_index = dict([(char, i + 1) for i, char in enumerate(target_tokens)])

input_vocab_size = len(input_token_index) + 1
target_vocab_size = len(target_token_index) + 1

# Encode sentences into sequences of integers
def encode_text(text, token_index, max_len):
    sequence = [token_index[word] for word in text.split() if word in token_index]
    sequence = sequence + [0] * (max_len - len(sequence))  # Pad sequences
    return sequence

# Determine maximum lengths for padding
max_encoder_seq_length = max([len(text.split()) for text in input_texts])
max_decoder_seq_length = max([len(text.split()) for text in target_texts])

# Prepare the data
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length), dtype="int32")
decoder_input_data = np.zeros((len(target_texts), max_decoder_seq_length), dtype="int32")
decoder_target_data = np.zeros((len(target_texts), max_decoder_seq_length, target_vocab_size), dtype="float32")

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    encoder_input_data[i] = encode_text(input_text, input_token_index, max_encoder_seq_length)
    decoder_input_data[i] = encode_text(target_text, target_token_index, max_decoder_seq_length)
    for t, word in enumerate(target_text.split()):
        if word in target_token_index:
            decoder_target_data[i, t, target_token_index[word]] = 1.0
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[word]] = 1.0  # Shifts the target sequence

# Model definition
# Encoder
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(input_dim=input_vocab_size, output_dim=latent_dim)(encoder_inputs)
encoder_outputs, state_h, state_c = LSTM(latent_dim, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(input_dim=target_vocab_size, output_dim=latent_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(target_vocab_size, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Seq2Seq model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer=RMSprop(), loss="categorical_crossentropy", metrics=["accuracy"])

# Train the model
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2
)

# Inference models
# Encoder model for inference
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder model for inference
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_embedding, initial_state=decoder_states_inputs
)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

# Translation function
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1))
    start_token_index = 1 if target_token_index else 0  # Default to 0 if empty
    target_seq[0, 0] = start_token_index

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        reverse_target_token_index = dict((i, char) for char, i in target_token_index.items())
        sampled_word = reverse_target_token_index[sampled_token_index]
        decoded_sentence += ' ' + sampled_word

        # Exit condition: either hit max length or find stop character.
        if (sampled_word == '\n' or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (length 1).
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence.strip()

# Test the translation
test_input_seq = np.array(encode_text("hello", input_token_index, max_encoder_seq_length)).reshape(1, -1)
translated_sentence = decode_sequence(test_input_seq)
print("Translated sentence:", translated_sentence)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step - accuracy: 0.2000 - loss: 3.3388 - val_accuracy: 0.0000e+00 - val_loss: 1.7745
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step - accuracy: 0.4000 - loss: 3.2938 - val_accuracy: 0.0000e+00 - val_loss: 1.7887
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step - accuracy: 0.4000 - loss: 3.2541 - val_accuracy: 0.0000e+00 - val_loss: 1.8030
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - accuracy: 0.4667 - loss: 3.2131 - val_accuracy: 0.0000e+00 - val_loss: 1.8186
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - accuracy: 0.4667 - loss: 3.1678 - val_accuracy: 0.0000e+00 - val_loss: 1.8363
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - accuracy: 0.4667 - loss: 3.1157 - val_accuracy: 0.0000e+00 - val_loss: 1.8568
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 0.4667 - loss: 3.0544 - val_accuracy: 0.0000e+00 - val_loss: 1.8809
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - accuracy: 0.4667 - loss: 2.9810 - v